# Анализ базы данных сервиса для чтения книг по подписке.
Финальный проект по SQL


ЦЕЛЬ:Проанализировать базу данных для формулировки ценностного предложения для нового продукта.

ОПИСАНИЕ: Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

ДАННЫЕ:
Таблица books
Содержит данные о книгах:
book_id — идентификатор книги;
author_id — идентификатор автора;
title — название книги;
num_pages — количество страниц;
publication_date — дата публикации книги;
publisher_id — идентификатор издателя.
Таблица authors
Содержит данные об авторах:
author_id — идентификатор автора;
author — имя автора.
Таблица publishers
Содержит данные об издательствах:
publisher_id — идентификатор издательства;
publisher — название издательства;
Таблица ratings
Содержит данные о пользовательских оценках книг:
rating_id — идентификатор оценки;
book_id — идентификатор книги;
username — имя пользователя, оставившего оценку;
rating — оценка книги.
Таблица reviews
Содержит данные о пользовательских обзорах на книги:
review_id — идентификатор обзора;
book_id — идентификатор книги;
username — имя пользователя, написавшего обзор;
text — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})




## Изучим данные

Откроем таблицу books

In [2]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * 
           FROM books  
           LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Откроем таблицу authors

In [3]:
query = '''SELECT * 
           FROM authors 
           LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Откроем таблицу ratings

In [4]:
query = '''SELECT * 
           FROM ratings
           LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Откроем таблицу reviews

In [5]:
query = '''SELECT * 
           FROM reviews 
           LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Откроем таблицу publishers

In [6]:
query = '''SELECT * 
           FROM publishers 
           LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Исследовательский анализ

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''SELECT COUNT(book_id)
           FROM books
           WHERE  publication_date >'2000-01-01'
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


ВЫВОД:После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [8]:
query = '''SELECT b.title, 
           COUNT(DISTINCT re.review_id), 
           AVG(r.rating)
           FROM books AS b
           LEFT JOIN reviews AS re ON b.book_id = re.book_id
           LEFT JOIN  ratings AS r ON b.book_id = r.book_id
           GROUP BY b.book_id
           ORDER BY COUNT(re.review_id) DESC
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,title,count,avg
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...
995,Essential Tales and Poems,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


ВЫВОД: Топ-3:
- 1120 обзоров на книгу "Twilight (Twilight #1)"Сумерки" — роман Стефани Майер со средней оценкой 3.66.
- 528 обзоров на книгу The Hobbit or There and Back Again «Хоббит, или Туда и обратно» — детский фантастический роман английского писателя J. R. R. Tolkien со средней оценкой 4.12
- 516 обзоров на книгу The Catcher in the Rye «Над пропастью во ржи» — американский роман Дж. Д. Сэлинджера со средней оценкой 3.82



### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
query = '''SELECT p.publisher,
           COUNT(b.book_id) AS cnt_book
           FROM publishers AS p
           JOIN books AS b ON p.publisher_id = b.publisher_id
           WHERE num_pages>50 
           GROUP BY p.publisher
           ORDER BY cnt_book DESC
           LIMIT 1    
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,cnt_book
0,Penguin Books,42


ВЫВОД:Издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
query = '''SELECT a.author,
           AVG(r.rating)
           FROM (SELECT book_id,
           COUNT(rating_id) 
           FROM ratings 
           GROUP BY book_id
           HAVING COUNT(rating_id)>50
           ORDER BY COUNT(rating_id) DESC) AS ratin
           JOIN books AS b ON ratin.book_id = b.book_id     
           JOIN authors AS a ON b.author_id = a.author_id
           JOIN ratings AS r ON b.book_id = r.book_id
           GROUP BY a.author
           ORDER BY AVG(r.rating) DESC
           LIMIT 10    
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


ВЫВОД: Автор J.K. Rowling/Mary GrandPré с самой высокой средней оценкой книг (средним рейтинг 4.28) (Известная по серии романов о Гарри Поттере.)

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = '''SELECT AVG(count_rating.rev)
           FROM(SELECT COUNT(review_id) AS rev,
  username
  FROM reviews
  GROUP BY username) AS count_rating
  WHERE username IN (SELECT username
           FROM ratings
           GROUP BY username
           HAVING  COUNT(rating_id)>48
           )     
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок- 24.

## Вывод

ВЫВОД:

После 1 января 2000 года вышло 819 книг. Британское издательство "Penguin Books" , основанное в 1935 году в Лондоне  выпустило наибольшее число книг толще 50 страниц. 
Топ-3:
1120 обзоров на книгу "Twilight (Twilight #1)"Сумерки" — роман Стефани Майер со средней оценкой 3.66.
528 обзоров на книгу The Hobbit or There and Back Again «Хоббит, или Туда и обратно» — детский фантастический роман английского писателя J. R. R. Tolkien со средней оценкой 4.12
516 обзоров на книгу The Catcher in the Rye «Над пропастью во ржи» — американский роман Дж. Д. Сэлинджера со средней оценкой 3.82
Автор J.K. Rowling/Mary GrandPré с самой высокой средней оценкой книг (средним рейтинг 4.28) (Известная по серии романов о Гарри Поттере.)
Среднее количество обзоров 24 от пользователей, которые поставили больше 48 оценок.


